In [ ]:
# =================== Imports ===================
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import KFold

D1

In [ ]:
# =================== Load & Split Data ===================
df = pd.read_csv("fulldata_rd1.csv")
print(df.info)
selected_features = [ 'u', 'g', 'r', 'i', 'z', 'petroRad_u',
    'petroRad_g', 'petroRad_i', 'petroRad_r', 'petroRad_z', 'petroFlux_u',
    'petroFlux_g', 'petroFlux_i', 'petroFlux_r', 'petroFlux_z',
    'petroR50_u', 'petroR50_g', 'petroR50_i', 'petroR50_r', 'petroR50_z',
    'psfMag_u', 'psfMag_r', 'psfMag_g', 'psfMag_i', 'psfMag_z', 'expAB_u',
    'expAB_g', 'expAB_r', 'expAB_i', 'expAB_z', 'u_g', 'g_r', 'r_i', 'i_z']

X = df[selected_features].values
y = df['redshift'].values

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

<bound method DataFrame.info of                  ra        dec         u         g         r         i  \
0        131.598399  49.892824  19.91396  19.88652  19.55724  19.48510   
1        131.614363  49.895224  21.72610  19.82969  18.97453  18.70725   
2        131.505227  49.973403  20.60903  18.26406  17.05696  16.70153   
3        131.648792  49.892616  21.15924  19.47403  18.52742  18.26878   
4        131.507923  50.000914  20.59268  20.20429  20.11275  20.12046   
...             ...        ...       ...       ...       ...       ...   
3433450  357.987510  31.056477  22.23353  21.64426  20.20037  19.35161   
3433451  358.289234  30.883538  21.16952  19.15517  17.89892  17.31643   
3433452  358.192615  30.995745  21.53314  21.58024  21.21893  21.08595   
3433453  358.214519  30.953510  26.16734  21.76876  20.14158  19.14905   
3433454  358.260920  31.025976  25.62514  23.08200  21.21624  19.88421   

                z  petroRad_u  petroRad_g  petroRad_i  ...   expAB_u  \
0      

Linear Regressor

In [ ]:
from sklearn.preprocessing import StandardScaler

# ========= Normalize features =========
scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)

# Convert to torch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# ========= Define RMSE function =========
def rmse_torch(y_true, y_pred):
    return torch.sqrt(torch.mean((y_true - y_pred) ** 2)).item()

# ========= Single run utility =========
def run_single_model(model, model_name, X_tensor, y_tensor, test_size=0.2, random_state=42):
    print(f"\n🔍 Running {model_name} on a single train-test split...")

    X_train, X_test, y_train, y_test = train_test_split(
        X_tensor, y_tensor, test_size=test_size, random_state=random_state
    )

    # Fit model
    model.fit(X_train.numpy(), y_train.numpy().ravel())

    # Predict
    preds = model.predict(X_test.numpy())
    preds_tensor = torch.tensor(preds, dtype=torch.float32).reshape(-1, 1)

    # Evaluate
    rmse_val = rmse_torch(y_test, preds_tensor)
    print(f"📊 {model_name} RMSE on test set: {rmse_val:.5f}")

# ========== Linear Regression Single Run ==========
run_single_model(LinearRegression(), "Linear Regression", X_tensor, y_tensor)



🔍 Running Linear Regression on a single train-test split...
📊 Linear Regression RMSE on test set: 0.42109


RidgeRegressor

In [ ]:
# =================== Imports ===================
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# =================== Normalize Features ===================
scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)

# =================== Split Data ===================
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# =================== Define and Train Model ===================
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train, y_train)

# =================== Predictions ===================
y_train_pred = ridge_model.predict(X_train)
y_test_pred = ridge_model.predict(X_test)

# =================== Evaluation ===================
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# =================== Results ===================
print("📊 Ridge Regression Results:")
print(f"Train RMSE: {train_rmse:.5f}")
print(f"Test RMSE:  {test_rmse:.5f}")
print(f"Train R²:   {train_r2:.5f}")
print(f"Test R²:    {test_r2:.5f}")


📊 Ridge Regression Results:
Train RMSE: 0.31040
Test RMSE:  0.31068
Train R²:   0.48419
Test R²:    0.48294


Lasso Regression

In [ ]:
# =================== Imports ===================
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

# =================== Normalize Features ===================
scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)

# =================== Split Data ===================
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# =================== Define Model ===================
lasso_model = Lasso(alpha=0.01, random_state=42)

# =================== Train Model ===================
lasso_model.fit(X_train, y_train)

# =================== Predictions ===================
y_train_pred = lasso_model.predict(X_train)
y_test_pred = lasso_model.predict(X_test)

# =================== Evaluation ===================
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# =================== Results ===================
print("📊 Lasso Regression Results:")
print(f"Train RMSE: {train_rmse:.5f}")
print(f"Test RMSE:  {test_rmse:.5f}")
print(f"Train R²:   {train_r2:.5f}")
print(f"Test R²:    {test_r2:.5f}")


📊 Lasso Regression Results:
Train RMSE: 0.41295
Test RMSE:  0.41271
Train R²:   0.08706
Test R²:    0.08755


ElasticNetRegressor

In [ ]:
# =================== Imports ===================
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

# =================== Normalize Features ===================
scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)

# =================== Split Data ===================
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# =================== Define Model ===================
elastic_model = ElasticNet(alpha=0.01, l1_ratio=0.5, random_state=42)

# =================== Train Model ===================
elastic_model.fit(X_train, y_train)

# =================== Predictions ===================
y_train_pred = elastic_model.predict(X_train)
y_test_pred = elastic_model.predict(X_test)

# =================== Evaluation ===================
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# =================== Results ===================
print("📊 ElasticNet Regression Results:")
print(f"Train RMSE: {train_rmse:.5f}")
print(f"Test RMSE:  {test_rmse:.5f}")
print(f"Train R²:   {train_r2:.5f}")
print(f"Test R²:    {test_r2:.5f}")


📊 ElasticNet Regression Results:
Train RMSE: 0.41196
Test RMSE:  0.41188
Train R²:   0.09140
Test R²:    0.09123


D2

In [ ]:
# =================== Load & Split Data ===================
df = pd.read_csv("fulldata_rd2.csv") 
print(df.info)
selected_features = [ 'u', 'g', 'r', 'i', 'z', 'petroRad_u',
    'petroRad_g', 'petroRad_i', 'petroRad_r', 'petroRad_z', 'petroFlux_u',
    'petroFlux_g', 'petroFlux_i', 'petroFlux_r', 'petroFlux_z',
    'petroR50_u', 'petroR50_g', 'petroR50_i', 'petroR50_r', 'petroR50_z',
    'psfMag_u', 'psfMag_r', 'psfMag_g', 'psfMag_i', 'psfMag_z', 'expAB_u',
    'expAB_g', 'expAB_r', 'expAB_i', 'expAB_z', 'u_g', 'g_r', 'r_i', 'i_z']

X = df[selected_features].values
y = df['redshift'].values

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

<bound method DataFrame.info of                  ra        dec         u         g         r         i  \
0        131.598399  49.892824  19.91396  19.88652  19.55724  19.48510   
1        131.614363  49.895224  21.72610  19.82969  18.97453  18.70725   
2        131.505227  49.973403  20.60903  18.26406  17.05696  16.70153   
3        131.648792  49.892616  21.15924  19.47403  18.52742  18.26878   
4        131.507923  50.000914  20.59268  20.20429  20.11275  20.12046   
...             ...        ...       ...       ...       ...       ...   
3697850  358.209126  31.084833  21.95207  21.40699  21.20043  21.05200   
3697851  358.209471  31.005526  22.50246  21.91592  21.85252  21.72241   
3697852  358.214519  30.953510  26.16734  21.76876  20.14158  19.14905   
3697853  358.260920  31.025976  25.62514  23.08200  21.21624  19.88421   
3697854  358.314917  30.920003  23.21627  21.22459  20.93306  20.91307   

                z  petroRad_u  petroRad_g  petroRad_i  ...   expAB_u  \
0      

Linear Regression

In [ ]:
from sklearn.preprocessing import StandardScaler

# ========= Normalize features =========
scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)

# Convert to torch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# ========= Define RMSE function =========
def rmse_torch(y_true, y_pred):
    return torch.sqrt(torch.mean((y_true - y_pred) ** 2)).item()

# ========= Single run utility =========
def run_single_model(model, model_name, X_tensor, y_tensor, test_size=0.2, random_state=42):
    print(f"\n🔍 Running {model_name} on a single train-test split...")

    X_train, X_test, y_train, y_test = train_test_split(
        X_tensor, y_tensor, test_size=test_size, random_state=random_state
    )

    # Fit model
    model.fit(X_train.numpy(), y_train.numpy().ravel())

    # Predict
    preds = model.predict(X_test.numpy())
    preds_tensor = torch.tensor(preds, dtype=torch.float32).reshape(-1, 1)

    # Evaluate
    rmse_val = rmse_torch(y_test, preds_tensor)
    print(f"📊 {model_name} RMSE on test set: {rmse_val:.5f}")

# ========== Linear Regression Single Run ==========
run_single_model(LinearRegression(), "Linear Regression", X_tensor, y_tensor)



🔍 Running Linear Regression on a single train-test split...
📊 Linear Regression RMSE on test set: 0.68391


RidgeRegressor

In [ ]:
# =================== Imports ===================
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# =================== Normalize Features ===================
scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)

# =================== Split Data ===================
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# =================== Define and Train Model ===================
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train, y_train)

# =================== Predictions ===================
y_train_pred = ridge_model.predict(X_train)
y_test_pred = ridge_model.predict(X_test)

# =================== Evaluation ===================
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# =================== Results ===================
print("📊 Ridge Regression Results:")
print(f"Train RMSE: {train_rmse:.5f}")
print(f"Test RMSE:  {test_rmse:.5f}")
print(f"Train R²:   {train_r2:.5f}")
print(f"Test R²:    {test_r2:.5f}")


📊 Ridge Regression Results:
Train RMSE: 0.52049
Test RMSE:  0.52051
Train R²:   0.43832
Test R²:    0.43797


Lasso Regression

In [ ]:
# =================== Imports ===================
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

# =================== Normalize Features ===================
scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)

# =================== Split Data ===================
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# =================== Define Model ===================
lasso_model = Lasso(alpha=0.01, random_state=42)

# =================== Train Model ===================
lasso_model.fit(X_train, y_train)

# =================== Predictions ===================
y_train_pred = lasso_model.predict(X_train)
y_test_pred = lasso_model.predict(X_test)

# =================== Evaluation ===================
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# =================== Results ===================
print("📊 Lasso Regression Results:")
print(f"Train RMSE: {train_rmse:.5f}")
print(f"Test RMSE:  {test_rmse:.5f}")
print(f"Train R²:   {train_r2:.5f}")
print(f"Test R²:    {test_r2:.5f}")


📊 Lasso Regression Results:
Train RMSE: 0.66265
Test RMSE:  0.66280
Train R²:   0.08960
Test R²:    0.08867


Elastic Net Regression

In [ ]:
# =================== Imports ===================
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

# =================== Normalize Features ===================
scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)

# =================== Split Data ===================
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# =================== Define Model ===================
elastic_model = ElasticNet(alpha=0.01, l1_ratio=0.5, random_state=42)

# =================== Train Model ===================
elastic_model.fit(X_train, y_train)

# =================== Predictions ===================
y_train_pred = elastic_model.predict(X_train)
y_test_pred = elastic_model.predict(X_test)

# =================== Evaluation ===================
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# =================== Results ===================
print("📊 ElasticNet Regression Results:")
print(f"Train RMSE: {train_rmse:.5f}")
print(f"Test RMSE:  {test_rmse:.5f}")
print(f"Train R²:   {train_r2:.5f}")
print(f"Test R²:    {test_r2:.5f}")


📊 ElasticNet Regression Results:
Train RMSE: 0.65895
Test RMSE:  0.65918
Train R²:   0.09974
Test R²:    0.09860
